# 네이버 뉴스 크롤링

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
import time
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH ='~/Desktop/'  #결과 저장할 경로 (바탕화면에 저장됨)

now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        #print(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  

        pattern = '(\d\w+\s\전)' #정규표현식 
 
        r = re.compile(pattern)
        match = r.search(test).group(0)
        #print(match)
        date_text.append(match)
   

#내용 정제화 함수

def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        # url 주소에다 날짜를 입력할 예정
        
        response = requests.get(url)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
        #print(soup)
        #<a>태그에서 제목과 링크주소 추출
        
        atags = soup.select('.news_tit') # 뉴스기사의 제목 셀렉트
        
        for atag in atags:
            title_text.append(atag.text)     #제목 따오기
            link_text.append(atag['href'])   #링크주소 따오기
        
        #신문사 추출
        
        source_lists = soup.select('.press')
        
        for source_list in source_lists:
            if source_list.i:
                source_list.i.extract()
            if source_list.find('span') == None:
                continue
            source_text.append(source_list.text)    #신문사
            
        #날짜 추출 
        
        date_lists = soup.select('.info_group')
        #print(date_lists)
        
        for date_list in date_lists:
            if date_list.i:
                date_list.i.extract()
            test=date_list.text   
            
            date_cleansing(test)  #날짜 정제 함수사용 
            

        #본문요약본
        contents_lists = soup.select('.dsc_txt_wrap')
        for contents_list in contents_lists:
            #print('==='*40)
            #print(contents_list)
            contents_cleansing(contents_list) #본문요약 정제화
        
        
        #모든 리스트 딕셔너리형태로 저장
        
        # print(len(date_text), len(title_text), len(source_text), len(contents_text), len(link_text))
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        
        #print(page)
        
        #print(result)
        df = pd.DataFrame(result)  #df로 변환
        page += 10
        #df
    
    
    # 새로 만들 파일이름 지정
    #outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    
    outputFileName = query + ' %s-%s-%s  %s시 %s분.xlsx' % (now.year, now.month, now.day, now.hour, now.minute)
    
    df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
    
    
    

def main():
    
    
    query = input("검색어 입력: ")  
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
    
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ") 
    s_date = input("시작날짜 입력(2020.02.18):") 
    e_date = input("끝날짜 입력(2020.02.18):")   
    
    
    crawler(maxpage,query,sort,s_date,e_date)
    
    print('크롤링 중입니다.')
    time.sleep(0.2) # 오류 방지
    print('저장이 완료되었습니다.')
    
main()

검색어 입력: 대한민국 야식
최대 크롤링할 페이지 수 입력하시오: 3
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2020.02.18):2020.02.18
끝날짜 입력(2020.02.18):2020.02.20
크롤링 중입니다.
저장이 완료되었습니다.
